<a href="https://colab.research.google.com/github/JavaFXpert/Qiskit2Intro/blob/main/qiskit2intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Qiskit 2

**Audience:** Learners with linear algebra and basic quantum mechanics background  
**Goal:** Learn quantum computing concepts *and* Qiskit syntax through guided, runnable examples.
  
> We'll build intuition first, then code each idea with Qiskit 2.x primitives and visualizations.


## 0. Setup

We'll install and import what we need from Qiskit.

In [ ]:
!pip install qiskit[visualization] qiskit-ibm-runtime qiskit-aer

In [ ]:
# Core Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, SparsePauliOp, Pauli

# Visualization (works with matplotlib)
from qiskit.visualization import plot_bloch_multivector, plot_histogram

from qiskit_aer import AerSimulator

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator, QiskitRuntimeService

import numpy as np

## 1. Qubits, Basis States, and Pauli Operators

**Concept primer.** A single qubit lives in a 2‑dimensional complex vector space spanned by computational basis states $\lvert 0\rangle$ and $\lvert 1\rangle$.  
The **Pauli operators** $X, Y, Z$ generate $\pi$ rotations about their respective axes on the Bloch sphere. Global phase is physically unobservable, while *relative* phase matters. You may also visualize single qubit states in the [Grokking the Bloch Sphere](https://javafxpert.github.io/grok-bloch/) app.

In [ ]:
# Start with |0>, apply Pauli rotations, and view the resulting statevector

# Create the statevector
sv = Statevector.from_label('0')  # |0>

# Create Pauli operators
X = Pauli('X')
Y = Pauli('Y')
Z = Pauli('Z')

# Apply the operators
sv_x = sv.evolve(X)
sv_y = sv.evolve(Y)
sv_z = sv.evolve(Z)

print("|0>  ->", sv.data)
print("X|0> ->", sv_x.data)   # [0, 1]
print("Y|0> ->", sv_y.data)   # [0, i]
print("Z|0> ->", sv_z.data)   # [1, 0]

In [ ]:
# Visualize |+> and |-> on the Bloch sphere to see *relative* phase
plus = Statevector([1/np.sqrt(2), 1/np.sqrt(2)])
minus = Statevector([1/np.sqrt(2), -1/np.sqrt(2)])

plot_bloch_multivector(plus)  # |+>

In [ ]:
plot_bloch_multivector(minus)  # |->

> **Concept Check:** Why does $Z\lvert +\rangle = \lvert -\rangle$ change the Bloch **longitude** but not move the state off the equator?

## 2. Single‑Qubit Gates: H, S, T, and Phase

**Concept primer.** The **Hadamard** gate creates equal superposition; **S** and **T** add phase shifts.  
$H$ maps $\lvert 0\rangle \mapsto \lvert +\rangle$ and $\lvert 1\rangle \mapsto \lvert -\rangle$.

**Note:** S = $R_z(\pi/2)$, T = $R_z(\pi/4)$.

In [ ]:
qc = QuantumCircuit(1)
qc.h(0)
qc.s(0)        # phase pi/2
qc.t(0)        # phase pi/4
qc.draw('mpl')

In [ ]:
# Compare statevectors with and without global phase


# Create circuit properly (separate steps)
qc_h = QuantumCircuit(1)
qc_h.h(0)
sv_h = Statevector.from_instruction(qc_h)

# Assuming 'qc' is defined elsewhere in your code
sv_hst = Statevector.from_instruction(qc)

# Print rounded data
print("H|0>  :", np.round(sv_h.data, 3))
print("HST|0>:", np.round(sv_hst.data, 3))

# Plot visualization
plot_bloch_multivector(sv_hst)

> **Try (optional):** Replace `s; t` with a single `rz(theta)` and find a `theta` that matches the Bloch point above.

## 3. Superposition and Measurement Statistics

**Concept primer.** Measurement in $Z$ collapses amplitudes to probabilities $p(0)=|a|^2$, $p(1)=|b|^2$.  
We sample outcomes with **Sampler** and visualize histograms.

When we measure a quantum state |ψ⟩ = a|0⟩ + b|1⟩, Born's rule tells us that the probability of measuring outcome '0' is |a|² and the probability of measuring outcome '1' is |b|². This is one of the fundamental postulates of quantum mechanics, introduced by Max Born in 1926.

In [ ]:
# Create a quantum circuit with 1 qubit and 1 classical bit for storing measurement
qc = QuantumCircuit(1, 1)

# Apply Hadamard gate to qubit 0 to create superposition state |+⟩ = (|0⟩ + |1⟩)/√2
# This gives equal probability (50/50) of measuring 0 or 1
qc.h(0)

# Measure qubit 0 and store the result in classical bit 0
# This collapses the superposition to either |0⟩ or |1⟩
qc.measure(0, 0)

# Initialize the Aer quantum simulator backend
# AerSimulator is Qiskit's high-performance circuit simulator
backend = AerSimulator()

# Generate a pass manager to transpile the circuit for the backend
# optimization_level=1 applies light optimizations while preserving circuit structure
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

# Transpile the circuit to match the backend's instruction set architecture (ISA)
# This converts the logical circuit to a physical circuit executable on the backend
isa_qc = pm.run(qc)

# Create a Sampler primitive for running the circuit and collecting measurement statistics
# The Sampler handles circuit execution and statistical sampling
sampler = Sampler(mode=backend)

# Execute the circuit 2000 times to gather statistics
# Multiple shots are needed to observe the probabilistic nature of quantum measurements
job = sampler.run([isa_qc], shots=2000)

# Retrieve the results from the completed job
result = job.result()

# Extract the measurement counts from the classical register 'c'
# counts will be a dictionary like {'0': ~1000, '1': ~1000} for this circuit
counts = result[0].data.c.get_counts()

# Visualize the measurement statistics as a histogram
# Should show approximately equal bars for outcomes '0' and '1'
plot_histogram(counts)

## 4. Two Qubits: Tensor Products and Entanglement (Bell States)

**Concept primer.** Multi‑qubit states live in tensor product spaces. Entanglement creates correlations not factorizable as single‑qubit products.

In this example we create one of the four Bell states: $\lvert \Phi^+\rangle = (\lvert 00\rangle + \lvert 11\rangle)/\sqrt{2}$

In [ ]:
bell = QuantumCircuit(2, 2)
bell.h(0);
bell.cx(0, 1)
bell.measure([0,1], [0,1])
bell.draw('mpl')

In [ ]:
backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_bell = pm.run(bell)
sampler = Sampler(mode=backend)
job = sampler.run([isa_bell], shots=1024)

result = job.result()
counts = result[0].data.c.get_counts()
plot_histogram(counts)

## 5. Building and Visualizing Circuits

**Concept primer.** Circuits compose gates in time. Drawing styles aid communication and debugging.

In [ ]:
qc = QuantumCircuit(3, 3)
qc.h(0)
qc.cx(0,1)
qc.cx(1,2)
qc.barrier()
qc.measure([0,1,2],[0,1,2])

# Multiple drawer backends: 'mpl', 'text', 'latex'
qc.draw('mpl')

## 6. Parameterized Circuits

**Concept primer.** Parameterized gates (e.g., `rx(theta)`) support sweeps and variational algorithms (VQE, QAOA). Continuous parameters are useful, for example, with optimization landscapes.

This code demonstrates parameterized circuits by creating a quantum circuit with a symbolic parameter theta as a placeholder for the RY gate's rotation angle, rather than hard-coding a specific value. The circuit is compiled once into its hardware-compatible form (isa_pq), then reused with different theta values from 0 to 2π by creating (circuit, parameter_values) tuples. When the sampler executes, it binds each theta value at runtime and measures the circuit, showing how the probability of measuring |1⟩ varies according to sin²(θ/2) - from 0 at θ=0, peaking at 1 when θ=π, and returning to 0 at θ=2π. This approach is essential for variational algorithms like VQE and QAOA, where the same circuit structure must be evaluated repeatedly with different parameters during optimization, avoiding the overhead of recompiling for each parameter value.

In [ ]:
from qiskit.circuit import Parameter
theta = Parameter('θ')

pq = QuantumCircuit(1)
pq.ry(theta, 0)
pq.measure_all()

backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_pq = pm.run(pq)
sampler = Sampler(mode=backend)
thetas = np.linspace(0, 2*np.pi, 9)

# Create list of (circuit, parameter_values) tuples for V2 API
# PUB stands for Primitive Unified Bloc, which is a
# single unit of work for a quantum primitive
pub_list = [(isa_pq, {theta: t}) for t in thetas]

# Run the sampler with V2 API format
job = sampler.run(pub_list, shots=1000)
result = job.result()

# Collect P(1) estimates
p1 = []
for i in range(len(thetas)):
    # Get the result for this parameter value
    pub_result = result[i]
    # Get counts from the data
    counts = pub_result.data.meas.get_counts()
    # Calculate probability of '1'
    p1.append(counts.get('1', 0) / sum(counts.values()))

# Display results
print(list(zip(np.round(thetas, 2), np.round(p1, 3))))



## 7. Primitives: Sampler and Estimator

**Concept primer.** Qiskit **primitives** standardize how we request sampling (bitstring probabilities) and expectation values.

- **Sampler:** returns bitstring distributions given circuits.
- **Estimator:** returns expectation values of observables on states prepared by circuits.

In [ ]:
# This example demonstrates how to use the Estimator primitive to measure the
# expectation value of an observable (ZZ) on a Bell state quantum circuit.

# Create a Bell state circuit
bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)

# Define the observable to measure - ZZ measures correlation between qubits
# SparsePauliOp creates an operator from Pauli string notation
observable = SparsePauliOp('ZZ')

# Set up the backend simulator that the Estimator will use
backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_bell = pm.run(bell)

# Apply the circuit's layout to the observable to ensure proper qubit mapping
# This aligns the observable with how qubits were mapped during transpilation
isa_observable = observable.apply_layout(isa_bell.layout)

# Initialize the Estimator primitive with the specified backend
# The Estimator is designed to compute expectation values of observables
estimator = Estimator(mode=backend)

# Run the Estimator job with circuit-observable pairs
# The Estimator takes tuples of (circuit, observable) to evaluate
job = estimator.run([(isa_bell, isa_observable)])
result = job.result()

# Extract the expectation value from the first (and only) pub result
# pub_result contains the data for the circuit-observable pair
pub_result = result[0]
exp_val = pub_result.data.evs  # evs = expectation values

# For a Bell state |00⟩ + |11⟩, the ZZ expectation value should be 1
print(f'Expectation value for ZZ: {exp_val}')


## 10. Dynamic Circuits and Classical Control Flow

**Concept primer.** Mid‑circuit measurement and control flow enable feedback (e.g., quantum teleportation with conditional gates).

**Example:** Quantum teleportation transfers an unknown quantum state between distant qubits using entanglement and classical communication. The protocol creates an entangled Bell pair, performs a Bell measurement on the sender's qubits, then applies corrective gates on the receiver's qubit based on the measurement results. Dynamic circuits are crucial because they enable real-time conditional operations—the if_test statements allow the circuit to measure qubits and immediately apply X or Z corrections based on those results within a single execution. Without dynamic circuits, teleportation would require multiple separate circuit runs, breaking the protocol's coherence.


In [ ]:
# Create quantum and classical registers
qr = QuantumRegister(3, 'q')  # 3 qubits: q0 (state to teleport), q1 (Alice's Bell), q2 (Bob's Bell)
cr = ClassicalRegister(2, 'c')  # 2 classical bits for Bell measurement results
qc = QuantumCircuit(qr, cr)

# Step 1: Create the state to teleport (optional - for demonstration)
# Here we'll create an arbitrary state on q0
qc.ry(1.2, qr[0])  # Rotate Y to create |ψ⟩ = cos(0.6)|0⟩ + sin(0.6)|1⟩
qc.barrier()

# Step 2: Create Bell pair between Alice (q1) and Bob (q2)
qc.h(qr[1])         # Hadamard on Alice's qubit
qc.cx(qr[1], qr[2]) # CNOT to create entanglement
qc.barrier()

# Step 3: Bell measurement on Alice's side (q0 and q1)
qc.cx(qr[0], qr[1]) # CNOT with state to teleport as control
qc.h(qr[0])         # Hadamard on the state to teleport
qc.barrier()

# Step 4: Measure Alice's qubits
qc.measure(qr[0], cr[0])
qc.measure(qr[1], cr[1])
qc.barrier()

# Step 5: Apply corrections on Bob's side based on measurement results
# If cr[1] == 1, apply X gate (bit flip)
with qc.if_test((cr[1], 1)):
    qc.x(qr[2])

# If cr[0] == 1, apply Z gate (phase flip)
with qc.if_test((cr[0], 1)):
    qc.z(qr[2])

# Draw the circuit
qc.draw('mpl')

## Wrap‑Up

- You saw *state preparation → measurement → statistics* for single and multi‑qubit systems.
- You learned how to use **Sampler** and **Estimator**, parameterized circuits, and basic transpilation.
- You connected formal ideas (Born's rule, entanglement, Bell) to concrete Qiskit code.
- You saw how to implement dynamic circuits in Qiskit.
